# ECSE 415 - Assignment 5: Video Analysis

## Authors

| *Name*                         |*Student ID*|
|--------------------------------|------------|
| Ana Gordon                     | 261113440 |
| Mathias Nahuel Pacheco Lemina  | 261116679  |

## Introduction
The goal of this assignment is 

https://www.freecodecamp.org/news/how-to-detect-objects-in-images-using-yolov8/#heading-how-to-get-started-with-yolov8

## `0` Preparation

Run this in a terminal first

*python3 -m venv venv && source venv/bin/activate && python -m pip install --upgrade pip && python -m pip install -r requirements.txt*

In [107]:
!pip install ultralytics
# !git clone https://github.com/ultralytics/yolov5
# %cd yolov5
# %pip install -qr requirements.txt comet_ml

### `Imports`

In [ ]:
# Set environment vars BEFORE importing torch/CUDA initialization
import os
os.environ["TORCH_USE_CUDA_DSA"] = "1"

import torch

# Choose device --> for CUDA DSA (on pcuter with compatible GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [ ]:
# imports
# import pandas as pd
# import shutil
# from sklearn.model_selection import train_test_split
# import numpy as np
# import cv2
# import os
# from matplotlib import pyplot as plt
# from torch.utils.data import Dataset, DataLoader, random_split
# from torchvision import transforms
# import torch.nn as nn
# import torch.optim as optim
# from tqdm import tqdm
# from ultralytics import YOLO
# from torchvision.transforms import functional as F
# from skimage.feature import hog
# from sklearn.svm import SVC
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# from IPython.display import display
# import seaborn as sns
# import random
# import matplotlib.pyplot as plt
# import matplotlib.patches as patches
# from torchvision.models.detection import fasterrcnn_mobilenet_v3_large_fpn
# from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
# from torch.cuda.amp import autocast, GradScaler
# import warnings
# torch.device("cuda")

device(type='cuda')

### `Paths`

In [ ]:
data_path = './Road_Signs_Detection_Dataset/'

train_images_path = os.path.join(data_path, "train/images/")
train_labels_path = os.path.join(data_path, "train/labels/")
test_images_path = os.path.join(data_path, "test/images/")

sample_submission_path = "sample_submission.csv"


## `1` Data Preparation

## `2` Model Implementation

## `3` Model Evaluation

## `4` Prediction & Kaggle Competition